# Yahoo Finance (Python)

- https://pypi.org/project/yfinance/

In [1]:
# Libraries
import numpy as np
import pandas as pd

import re

import requests as rq

import yfinance as yf

from bs4 import BeautifulSoup as bs
from xml.etree import ElementTree as et

from skimpy import clean_columns

In [2]:
# nVidia stock
nvidia = yf.Ticker("NVDA")
nvidia

yfinance.Ticker object <NVDA>

In [5]:
nvidia_df = nvidia.history(period="max")
nvidia_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1999-01-22 00:00:00-05:00,0.040112,0.044767,0.035575,0.037605,2714688000,0.0,0.0
1999-01-25 00:00:00-05:00,0.040589,0.042021,0.037605,0.041545,510480000,0.0,0.0
1999-01-26 00:00:00-05:00,0.042021,0.042857,0.037724,0.038321,343200000,0.0,0.0
1999-01-27 00:00:00-05:00,0.038440,0.039395,0.036291,0.038202,244368000,0.0,0.0
1999-01-28 00:00:00-05:00,0.038202,0.038440,0.037843,0.038082,227520000,0.0,0.0


In [6]:
nvidia_df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2026-01-05 00:00:00-05:00,191.759995,193.630005,186.149994,188.119995,183529700,0.0,0.0
2026-01-06 00:00:00-05:00,190.520004,192.169998,186.820007,187.240005,176862600,0.0,0.0
2026-01-07 00:00:00-05:00,188.570007,191.369995,186.559998,189.110001,153543200,0.0,0.0
2026-01-08 00:00:00-05:00,189.110001,189.550003,183.710007,185.039993,172457000,0.0,0.0
2026-01-09 00:00:00-05:00,185.054993,186.339996,183.670105,185.940002,94867344,0.0,0.0
